In [4]:
from google.colab import files
uploaded = files.upload() 

Saving labeled_data.csv to labeled_data (1).csv


In [5]:
import pandas as pd
def _get_data():
        try:

            # csv_path =  'labeled_data.csv'
            # df = pd.read_csv(io.BytesIO(uploaded['labeled_data.csv']))
            data = pd.read_csv('labeled_data.csv', sep=',', header=None, error_bad_lines=False) # use pandas to read CSV
            data = data.dropna() # drop any rows with nans
            Y = data[5]
           
            # Y = [int(i) for i in Y.values]
            return data[6].values,Y.values
        except ValueError:
            return None, None
_get_data()

(array(["!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...",
        '!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!',
        '!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit',
        ...,
        'young buck wanna eat!!.. dat nigguh like I aint fuckin dis up again',
        'youu got wild bitches tellin you lies',
        '~~Ruffled | Ntac Eileen Dahlia - Beautiful color combination of pink, orange, yellow &amp; white. A Coll http://t.co/H0dYEBvnZB'],
       dtype=object), array(['2', '1', '1', ..., '1', '1', '2'], dtype=object))

In [6]:
import io
import os
import re
import string
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint
from sklearn.externals import joblib
from sklearn import preprocessing
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import pdb


import nltk



TAG_RE = re.compile(r'<[^>]+>')
EPOCHS = 10
BATCH_SIZE = 30


def _get_vocabulary(documents):
    return set(" ".join(documents).split())

def _get_tokenizer(documents):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(documents)
    return tokenizer

def _clean_documents(documents):
    # convert to list
    documents = documents.tolist()
    # clean docs
    print("Starting cleaning %s documents" % len(documents))
    documents = [_clean_doc(doc) for doc in documents]
    return documents

def _clean_doc( doc):
    # remove HTML tags
    doc = _clean_tags(doc)
    # replace all newlines and tabs
    doc = doc.replace('\\n', ' ').replace('\\r', ' ').replace('\\t', ' ')
    # add missing space after full stops and commas
    doc = re.sub(r'(?<=[.,])(?=[^\s])', r' ', doc)
    # create tokens
    tokens = word_tokenize(doc)
    # downcase
    tokens = [w.lower() for w in tokens]
    # regexp
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    stripped = [re_punc.sub('', w) for w in tokens]
    # remove non-alphabetic tokens
    words = [word for word in stripped if word.isalpha()]
    # remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    # reduce word to base
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in words]
    print('.', end='', flush=True)
    return " ".join(stemmed)

def _clean_tags( text):
    return TAG_RE.sub('', text)

def _get_model( vocabulary_size=1, input_length=1, load_weights=False):
    try:
        model = Sequential()
        model.add(Embedding(vocabulary_size, 16, input_length=input_length))
        # model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
        # model.add(Dropout(0.7))
        model.add(MaxPooling1D(5))
        model.add(LSTM(10))
        model.add(Dense(1, activation='sigmoid'))

        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
        return model
    except ValueError:
        return None

# def _get_data():
#     try:
#         X = pd.read_csv(io.BytesIO(uploaded['labeled_data.csv']), sep=',', header=None, error_bad_lines=False) # use pandas to read CSV
#         X = X.dropna() # drop any rows with nans

#         return X[6].values,X[5].values
#     except ValueError:
#         return None, None



/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [7]:
try:
  X, Y = _get_data()
  documents = _clean_documents(X)
except:
  nltk.download("stopwords")
  nltk.download('punkt')
  X, Y = _get_data()
  documents = _clean_documents(X)

Starting cleaning 24783 documents
......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [8]:
vocabulary = _get_vocabulary(documents)
vocab_size = len(vocabulary) + 1
tokenizer = _get_tokenizer(documents)
max_document_length = max([len(s.split()) for s in documents])
encoded = tokenizer.texts_to_sequences(documents)
X = sequence.pad_sequences(encoded, maxlen=max_document_length)
model = _get_model(vocab_size, max_document_length)







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import pipeline, feature_extraction, svm, metrics

def LSTM_model():
        model = Sequential()
        model.add(Embedding(vocab_size, 16, input_length=max_document_length))
        # model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
        model.add(Dropout(0.2))
        # model.add(MaxPooling1D(5))
        # model.add(LSTM(10, return_sequences = True))
        # model.add(Dropout(0.5, ))
        model.add(LSTM(10, return_sequences = True))
        model.add(Dropout(0.2))
        model.add(LSTM(10))
        model.add(Dropout(0.2))
        # model.add(Dense(3, activation='softmax'))
        # model.add(Dropout(0.5))
        model.add(Dense(3, activation="softmax"))
        # model.add(Activation('softmax'))
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

def classify(X, y):
    # self.split(X,y)
    cv = StratifiedKFold(n_splits=5)
    results = [0,0,0, 0,0,0,0,0,0]
    for train_idx, test_idx, in cv.split(X, y):
        X_train, y_train = X[train_idx], y[train_idx]
        X_test, y_test = X[test_idx], y[test_idx]
        model = LSTM_model()
        model.fit(X_train, y_train, validation_split=0.2, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)

        y_pred = model.predict_classes(X_test) 
        y_test = [int(i) for i in y_test]
        print(metrics.classification_report(y_test, y_pred))
        results = [x + y for x, y in zip(results , list( metrics.precision_recall_fscore_support(y_test, y_pred, average="macro")[:-1] + metrics.precision_recall_fscore_support(y_test, y_pred, average="weighted")[:-1]))]
        break
    print([round(x / 5, 2) for x in results ])
print("LSTM Classify")
for i in range (5,20):
  EPOCHS = i
  classify(X, Y)


LSTM Classify

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Train on 15860 samples, validate on 3966 samples
Epoch 1/5





15860/15860 [==============================] - 32s 2ms/step - loss: 0.5545 - acc: 0.8084 - val_loss: 0.3459 - val_acc: 0.8901
Epoch 2/5
15860/15860 [==============================] - 31s 2ms/step - loss: 0.3353 - acc: 0.8956 - val_loss: 0.3121 - val_acc: 0.8986
Epoch 3/5
15860/15860 [==============================] - 31s 2ms/step - loss: 0.2640 - acc: 0.9168 - val_loss: 0.3089 - val_acc: 0.8981
Epoch 4/5
15860/15860 [==============================] - 31s 2ms/step - loss: 0.2009 - acc: 0.9334 - val_loss: 0.3485 - val_acc: 0.8981
Epoch 5/5
15860/15860 [==============================] - 32s 2ms/step - loss: 0.1541 - acc: 0.9511 - val_loss: 0.3911 - val_acc: 0.8901
              precision    recall  f1-score   support

           0       0.30      0.19      0.23       286
           1       0.91 

KeyboardInterrupt: ignored

In [0]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import pipeline, feature_extraction, svm, metrics
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
def CNN_model():
        model = Sequential()
        model.add(Embedding(vocab_size, 100, input_length=max_document_length))
        model.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))
        model.add(Dropout(0.2))
        model.add(MaxPooling1D(4))
        model.add(Conv1D(filters=128, kernel_size=5,  activation='relu'))
        model.add(GlobalAveragePooling1D())
        # model.add(LSTM(10))
        # model.add(Dense(3, activation='softmax'))
        # model.add(Dropout(0.5))
        model.add(Dense(3, activation="softmax"))
        # model.add(Activation('softmax'))
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

def classify(X, y):
    # self.split(X,y)
    cv = StratifiedKFold(n_splits=5)
    results = [0,0,0, 0,0,0,0,0,0]
    for train_idx, test_idx, in cv.split(X, y):
        X_train, y_train = X[train_idx], y[train_idx]
        X_test, y_test = X[test_idx], y[test_idx]
        model = CNN_model()
        model.fit(X_train, y_train, validation_split=0.25, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)

        y_pred = model.predict_classes(X_test)
        y_test = [int(i) for i in y_test]
        print(metrics.classification_report(y_test, y_pred))
        results = [x + y for x, y in zip(results , list( metrics.precision_recall_fscore_support(y_test, y_pred, average="macro")[:-1] + metrics.precision_recall_fscore_support(y_test, y_pred, average="weighted")[:-1]))]
        break
    print([round(x , 2) for x in results ])
for i in range (5,20，3):
  EPOCHS = i
  classify(X, Y)


In [0]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import pipeline, feature_extraction, svm, metrics
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
def CNN_LSTM_model():
        model = Sequential()
        model.add(Embedding(vocab_size, 100, input_length=max_document_length))
        model.add(Dropout(0.2))
        model.add(Conv1D(filters=100, kernel_size=4, activation='linear'))
        # model.add(Dropout(0.7))
        model.add(MaxPooling1D(4))
        model.add(LSTM(100, return_sequences = True))
      
        model.add(GlobalAveragePooling1D(data_format='channels_last'))
        # model.add(LSTM(10))
        # model.add(Dense(3, activation='softmax'))
        # model.add(Dropout(0.5))
       
        model.add(Dense(3, activation="softmax"))
        
        # model.add(Activation('softmax'))
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
        return model

def classify(X, y):
    # self.split(X,y)
    cv = StratifiedKFold(n_splits=5)
    results = [0,0,0, 0,0,0,0,0,0]
    for train_idx, test_idx, in cv.split(X, y):
        X_train, y_train = X[train_idx], y[train_idx]
        X_test, y_test = X[test_idx], y[test_idx]
        model = CNN_LSTM_model()
        model.fit(X_train, y_train, validation_split=0.25, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)

        y_pred = model.predict_classes(X_test)
        y_test = [int(i) for i in y_test]
        print(metrics.classification_report(y_test, y_pred))
        results = [x + y for x, y in zip(results , list( metrics.precision_recall_fscore_support(y_test, y_pred, average="macro")[:-1] + metrics.precision_recall_fscore_support(y_test, y_pred, average="weighted")[:-1]))]
        break
    print([round(x , 2) for x in results ])
for i in range (5, 20, 3):
  EPOCHS = i
  classify(X, Y)


Train on 14869 samples, validate on 4957 samples
Epoch 1/5
14869/14869 [==============================] - 32s 2ms/step - loss: 0.4863 - acc: 0.8340 - val_loss: 0.2892 - val_acc: 0.9112
Epoch 2/5
14869/14869 [==============================] - 30s 2ms/step - loss: 0.2791 - acc: 0.9106 - val_loss: 0.2796 - val_acc: 0.9135
Epoch 3/5
14869/14869 [==============================] - 30s 2ms/step - loss: 0.2040 - acc: 0.9309 - val_loss: 0.3055 - val_acc: 0.9020
Epoch 4/5
14869/14869 [==============================] - 30s 2ms/step - loss: 0.1318 - acc: 0.9562 - val_loss: 0.3449 - val_acc: 0.8882
Epoch 5/5
14869/14869 [==============================] - 30s 2ms/step - loss: 0.0765 - acc: 0.9780 - val_loss: 0.3705 - val_acc: 0.8901
              precision    recall  f1-score   support

           0       0.33      0.34      0.34       286
           1       0.91      0.92      0.92      3838
           2       0.81      0.77      0.79       833

    accuracy                           0.86      4957

In [0]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import pipeline, feature_extraction, svm, metrics
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

def CNN_LSTM_model():
        model = Sequential()
        model.add(Embedding(vocab_size, 100, input_length=max_document_length))
        model.add(Dropout(0.2))
        model.add(Conv1D(filters=100, kernel_size=4, activation='linear'))
        # model.add(Dropout(0.7))
        model.add(MaxPooling1D(4))
        model.add(LSTM(100, return_sequences = True))
      
        model.add(GlobalAveragePooling1D(data_format='channels_last'))
        # model.add(LSTM(10))
        # model.add(Dense(3, activation='softmax'))
        # model.add(Dropout(0.5))
       
        model.add(Dense(3, activation="softmax"))
        
        # model.add(Activation('softmax'))
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
        return model

def classify(X, y):
    # self.split(X,y)
    cv = StratifiedKFold(n_splits=5)
    results = [0,0,0, 0,0,0,0,0,0]
    for train_idx, test_idx, in cv.split(X, y):
        X_train, y_train = X[train_idx], y[train_idx]
        X_test, y_test = X[test_idx], y[test_idx]
        model = CNN_LSTM_model()
        model.fit(X_train, y_train, validation_split=0.25, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)

        y_pred = model.predict_classes(X_test)
        y_test = [int(i) for i in y_test]
        print(metrics.classification_report(y_test, y_pred))
        results = [x + y for x, y in zip(results , list( metrics.precision_recall_fscore_support(y_test, y_pred, average="macro")[:-1] + metrics.precision_recall_fscore_support(y_test, y_pred, average="weighted")[:-1]))]
        break
    print([round(x , 2) for x in results ])
for i in range (5, 20, 3):
  EPOCHS = i
  classify(X, Y)
